# 作为服务供中型团队使用

中型团队指那种使用人数高于20人且规模但低于100人的基本固定的团队.这种团队一般是算法研发方面独角兽初创公司或者大公司的特定方向研发团队.通常会拥有不止一台工作站.我们以一台8显卡的工作站为例,假设一个工作站最多可以让20个人使用,那么也就只需要5个这样的工作站就可以支撑一个100人的研发团队.让他们在统一的jupyterhub平台进行研发既可以避免相互干扰弄坏环境,又可以统一工具避免不必要的沟通成本.

这种场景下我们可以沿用小型团队的资源,只是改为docker swarm集群化部署.

部署的例子我放在[项目](https://github.com/hsz1273327/TutorialForPython/tree/master/%E5%B7%A5%E5%85%B7%E9%93%BE%E7%AF%87/%E4%BA%A4%E4%BA%92%E7%8E%AF%E5%A2%83jupyter/%E5%A4%9A%E7%94%A8%E6%88%B7%E6%9C%8D%E5%8A%A1/examples/middleteam)下

## swarm环境下部署jupyterhub

在部署jupyterhub到swarm上之前我们应该先规划下swarm的配置.通常manager节点有条件的话应该独立于高算力机器,仅用于管理集群和提供接口.而那些高算力机器则应该作为worker挂载在swarm集群中.当然没条件那就凑活部署了.

我们通常将hub服务部署在swarm的manager节点上,而用户的notebook容器则应该尽量部署在worker上以避免过重的资源占用影响集群稳定.

对于swarm方式部署,更加推荐将hub和[proxy](https://hub.docker.com/r/jupyterhub/configurable-http-proxy)拆开部署.需要注意无论是hub还是proxy,都只能部署一个实例.不过不用担心,下面是hub和
proxy在100~150人负载时的资源占用情况:

Component | CPU (mean/peak) | Memory (mean/peak)
---|---|---
Hub| 4% / 13%| (230 MB / 260 MB)
Proxy| 6% / 13%| (47 MB / 65 MB)

可以看出单个hub和proxy实例就足以支撑这个规模的负载.拆开部署则是提供一个进一步拆分负载以及容错的能力.

swarm中部署jupyterhub有两个选择:

+ 追求网络效率的部署方式--`proxy`使用`vip`模式的`host mode`方式部署,`hub`,`notebook`使用`dnsrr`模式部署.

+ 追求便捷性和容错的部署方式--`proxy`,`hub`,`notebook`都有使用`vip`模式部署


在单用户镜像方面,swarm部署和单机并无区别.我们一样可以通过设置后缀对应的不同的镜像来做到差异化镜像.不同之处只是swarm部署有节点限制概念.我们可以通过额外的节点限制让用cpu的镜像在cpu的机器上跑.用gpu的镜像在gpu机器上跑.

这边依然推荐使用[我的镜像](https://hub.docker.com/r/hsz1273327/jupyterhub-for-swarm)

可以参考下面的例子:

+ 先创建一个可以挂在的overlay网络

```bash
docker network create --driver overlay --attachable jupyterhub_network
```

+ `docker-compose.yml`

```yml
version: "3.8"
services:
    proxy:
        image: jupyterhub/configurable-http-proxy:4.5
        networks:
            - jupyterhub_network
        ports:
            - mode: host
              protocol: tcp
              published: 8001
              target: 8000
        environment:
            CONFIGPROXY_AUTH_TOKEN: abc123
        command:
            - configurable-http-proxy
            - "--error-target"
            - "http://jupyterhub/hub/error"

        deploy:
            mode: global
            placement:
                constraints:
                    - node.hostname==RaspPiNode1
            resources:
                limits:
                    cpus: '1.0'
                    memory: 500M
                reservations:
                    cpus: '0.1'
                    memory: 10M
            restart_policy:
                condition: on-failure
                delay: 5s
                max_attempts: 3
                window: 100s
        
    jupyterhub:
        image: hsz1273327/jupyterhub-for-swarm:4.0.1
        hostname: jupyterhub
        container_name: jupyterhub
        networks:
            - jupyterhub_network
        volumes:
            - "/var/run/docker.sock:/var/run/docker.sock"
            - "jupyterhub-data:/data"
            - "jupyterhub-notebook:/jupyterhub_data"
        environment:
            PROXY_SHOULD_START: "False"
            PROXY_API_URL: http://proxy:8001
            CONFIGPROXY_AUTH_TOKEN: abc123
            HUB_DB_URL: "postgresql://postgres:postgres@<pghost>/jupyterhub"
            SPAWNER_NETWORK_NAME: jupyterhub_network
            SPAWNER_NOTEBOOK_IMAGE: hsz1273327/data-analyse-notebook:notebook-6.5.4
            SPAWNER_ENDPOINT_MODE: dnsrr
            SPAWNER_PERSISTENCE_VOLUME_TYPE: nfs3
            SPAWNER_PERSISTENCE_NFS_HOST: <nfs_host>
            SPAWNER_PERSISTENCE_NFS_DEVICE: ":<nfs_device>"
            SPAWNER_CONSTRAINT_IMAGES: "jetson:node.labels.calculation_type==gpu->hsz1273327/gpu-torch-notebook:pytorch2.0.1-cuda11.8.0-notebook6.5.4;quant:node.labels.calculation_type==cpu->hsz1273327/quant-notebook:notebook-6.5.4"
            SPAWNER_CONSTRAINT_WITH_GPUS: "jetson->1"
        deploy:
            endpoint_mode: dnsrr
            mode: replicated
            replicas: 1
            placement:
                constraints:
                    - node.role==manager
            resources:
                limits:
                    cpus: '2.0'
                    memory: 500M
                reservations:
                    cpus: '0.1'
                    memory: 10M
            restart_policy:
                condition: on-failure
                delay: 5s
                max_attempts: 3
                window: 100s

volumes:
    jupyterhub-data:
    jupyterhub-notebook:
        driver_opts:
          type: "nfs"
          o: "addr=<nfs_host>,rw,vers=3,nolock,soft"
          device: ":<nfs_device>"
networks:
    jupyterhub_network:
        external: true
```

有如下几个点需要注意:

1. `proxy`和`jupyterhub`的环境变量`CONFIGPROXY_AUTH_TOKEN`需要一致.
2. 强烈建议`HUB_DB_URL`中设置pg或者mysql等外部数据库.swarm的volume并不支持集群内同步
3. 强烈建议使用nfs保存用户的notebook资产,swarm的volume并不支持集群内同步